In [1]:
import dask.dataframe as dd
import dask_ml
import lightgbm as lgb
from distributed import Client, LocalCluster

In [2]:
cluster = LocalCluster(n_workers=4, threads_per_worker=2)
client = Client(cluster)

In [3]:
df = dd.read_csv("covtype/covtype.csv", blocksize="64MB")

In [4]:
df.shape

(Delayed('int-1d52fdac-81a2-470d-9aaa-4823040383e9'), 55)

In [5]:
df.shape[0].compute()

581012

In [6]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = dask_ml.model_selection.train_test_split(X, y)

In [19]:
dask_model = lgb.DaskLGBMClassifier(n_estimators=200, client=client)
dask_model.fit(X_train, y_train)

/home/avanwyk/.local/share/mambaforge/envs/lgbmenv/lib/python3.10/site-packages/lightgbm/dask.py:526: UserWarning: Parameter n_jobs will be ignored.
  _log_warning(f"Parameter {param_alias} will be ignored.")


Finding random open ports for workers
[LightGBM] [Info] Trying to bind port 59293...
[LightGBM] [Info] Binding port 59293 succeeded
[LightGBM] [Info] Listening...
[LightGBM] [Info] Listening...
[LightGBM] [Info] Connected to rank 0
[LightGBM] [Info] Local rank: 1, total number of machines: 2
[LightGBM] [Warning] num_threads is set=2, n_jobs=-1 will be ignored. Current value: num_threads=2


DaskLGBMClassifier(client=<Client: 'tcp://127.0.0.1:39171' processes=4 threads=8, memory=15.01 GiB>,
                   n_estimators=200, num_threads=2, time_out=120,
                   tree_learner='data')

In [22]:
predictions = dask_model.predict(X_test)

In [25]:
predictions.compute()

array([5., 5., 2., ..., 3., 3., 3.])

In [16]:
dask_model.score(X_test.to_dask_array(lengths=True), y_test.to_dask_array(lengths=True))